In [1]:
!pip install pyspark findspark

In [15]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')
import findspark
findspark.init()

In [3]:
print("hello nam")

hello nam


In [1]:
from pyspark.ml.classification import RandomForestClassificationModel
from pyspark.sql import SparkSession

# Khởi tạo SparkSession
spark = SparkSession.builder.appName("rfmodel").master("spark://spark-master:7077").getOrCreate()


In [2]:
df = spark.read.csv("hdfs://namenode:9000/data/spotify_data_2023.csv", header=True, inferSchema=True)
df.cache()
model_path = "hdfs://namenode:9000/model/random_forest_model"
rf_model = RandomForestClassificationModel.load(model_path)

In [5]:
selected_columns = [
    "acousticness",
    "danceability",
    "duration_ms",
    "energy",
    "instrumentalness",
    "liveness",
    "loudness",
    "mode",
    "speechiness",
    "tempo",
    "key",
    "time_signature",
    "valence",
    "release_year",
    "release_month",
    "followers"
]
df=df.select(*selected_columns)

In [6]:
df.printSchema()

root
 |-- acousticness: string (nullable = true)
 |-- danceability: string (nullable = true)
 |-- duration_ms: string (nullable = true)
 |-- energy: string (nullable = true)
 |-- instrumentalness: string (nullable = true)
 |-- liveness: string (nullable = true)
 |-- loudness: string (nullable = true)
 |-- mode: string (nullable = true)
 |-- speechiness: string (nullable = true)
 |-- tempo: double (nullable = true)
 |-- key: string (nullable = true)
 |-- time_signature: string (nullable = true)
 |-- valence: string (nullable = true)
 |-- release_year: string (nullable = true)
 |-- release_month: string (nullable = true)
 |-- followers: string (nullable = true)



In [3]:
df_202x=df.filter((df["release_year"]=="2023") | (df["release_year"]=="2022"))


In [4]:
df_202x.cache()
df_202x.count()

95239

DataFrame[acousticness: string, danceability: string, duration_ms: string, energy: string, instrumentalness: string, liveness: string, loudness: string, mode: string, speechiness: string, tempo: double, key: string, time_signature: string, valence: string, release_year: string, release_month: string, followers: string]

In [5]:
from pyspark.sql.functions import when, col
cols_to_cast = [
    "acousticness", "danceability", "duration_ms", "energy", "instrumentalness",
    "liveness", "loudness", "mode", "speechiness", "tempo", "key", 
    "time_signature", "valence", "release_year", "followers"
]
df_test = df_202x.select(*[col(c).cast("double").alias(c) for c in cols_to_cast ],col("release_month"))
df_test.count()

95239

375141

In [6]:
df_min = df_test.filter((df_test["duration_ms"] > 80000) & (df_test["duration_ms"] < 360000))
df_min.count()


84928

In [14]:
from pyspark.sql.functions import col, sum

df_null=df_min.select([sum(col(c).isNull().cast("int")).alias(c + "_nulls") for c in df_min.columns]).show()


+------------------+------------------+-----------------+------------+----------------------+--------------+--------------+----------+-----------------+-----------+---------+--------------------+-------------+------------------+---------------+-------------------+
|acousticness_nulls|danceability_nulls|duration_ms_nulls|energy_nulls|instrumentalness_nulls|liveness_nulls|loudness_nulls|mode_nulls|speechiness_nulls|tempo_nulls|key_nulls|time_signature_nulls|valence_nulls|release_year_nulls|followers_nulls|release_month_nulls|
+------------------+------------------+-----------------+------------+----------------------+--------------+--------------+----------+-----------------+-----------+---------+--------------------+-------------+------------------+---------------+-------------------+
|                 0|                 0|                0|           0|                     0|             0|             0|         0|                0|          0|        0|                   0|          

In [7]:
# months = ["January", "February", "March", "April", "May", "June",
#           "July", "August", "September", "October", "November", "December"]

# df_by_month_name = [df_min.filter(col("release_month") == m) for m in months]
df_min=df_min.filter(df_min["followers"]>100)

In [8]:

# feature_cols = [col for col in df.columns if col not in "class"]
# assembler = VectorAssembler(inputCols=feature_cols, outputCol="features_unscaled")
# df_vec = assembler.transform(df_test)
# scaler = StandardScaler(inputCol="features_unscaled",outputCol="features",withStd=True,withMean=True)
# scaler_model=scaler.fit(df_vec)
# df_scaled = scaler_model.transform(df_vec)
df_min.printSchema()

root
 |-- acousticness: double (nullable = true)
 |-- danceability: double (nullable = true)
 |-- duration_ms: double (nullable = true)
 |-- energy: double (nullable = true)
 |-- instrumentalness: double (nullable = true)
 |-- liveness: double (nullable = true)
 |-- loudness: double (nullable = true)
 |-- mode: double (nullable = true)
 |-- speechiness: double (nullable = true)
 |-- tempo: double (nullable = true)
 |-- key: double (nullable = true)
 |-- time_signature: double (nullable = true)
 |-- valence: double (nullable = true)
 |-- release_year: double (nullable = true)
 |-- followers: double (nullable = true)
 |-- release_month: string (nullable = true)



In [9]:
from pyspark.ml.feature import VectorAssembler, StandardScaler 
from pyspark.sql.functions import col
feature_cols=[col for col in df_min.columns if col not in ["followers","release_month","release_year"]]
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features_unscaled")
df_vec = assembler.transform(df_min)
scaler = StandardScaler(inputCol="features_unscaled",outputCol="features",withStd=True,withMean=True)
scaler_model=scaler.fit(df_vec)
df_scaled = scaler_model.transform(df_vec)
    

In [18]:
print(len(assembler.getInputCols()))


13


In [10]:
predictions = rf_model.transform(df_scaled)
predictions.select("prediction").show()

+----------+
|prediction|
+----------+
|      10.0|
|       6.0|
|       6.0|
|       9.0|
|      10.0|
|       6.0|
|       9.0|
|       9.0|
|       6.0|
|       5.0|
|      10.0|
|       6.0|
|       6.0|
|       5.0|
|      10.0|
|       5.0|
|       6.0|
|       5.0|
|       6.0|
|       9.0|
+----------+
only showing top 20 rows



In [24]:
df_min.count()

273677

In [11]:
predictions.groupBy("prediction").count().orderBy("count").show()
predictions.printSchema()

+----------+-----+
|prediction|count|
+----------+-----+
|       0.0|   60|
|       7.0|   94|
|       1.0|  180|
|       2.0|  353|
|       3.0|  703|
|       8.0| 2071|
|       9.0| 2997|
|       6.0|11406|
|      10.0|16891|
|       5.0|23897|
+----------+-----+

root
 |-- acousticness: double (nullable = true)
 |-- danceability: double (nullable = true)
 |-- duration_ms: double (nullable = true)
 |-- energy: double (nullable = true)
 |-- instrumentalness: double (nullable = true)
 |-- liveness: double (nullable = true)
 |-- loudness: double (nullable = true)
 |-- mode: double (nullable = true)
 |-- speechiness: double (nullable = true)
 |-- tempo: double (nullable = true)
 |-- key: double (nullable = true)
 |-- time_signature: double (nullable = true)
 |-- valence: double (nullable = true)
 |-- release_year: double (nullable = true)
 |-- followers: double (nullable = true)
 |-- release_month: string (nullable = true)
 |-- features_unscaled: vector (nullable = true)
 |-- features: v

In [12]:
predictions_final= predictions.drop("features_unscaled","features","rawPrediction","probability")

In [13]:
predictions_final.coalesce(1).write.mode("overwrite").option("header", "true").csv("hdfs://namenode:9000/data/prediction.csv")


In [27]:
predictions_final.show(5)

+------------+------------+-----------+------+----------------+--------+--------+----+-----------+-------+---+--------------+-------+------------+---------+-------------+----------+
|acousticness|danceability|duration_ms|energy|instrumentalness|liveness|loudness|mode|speechiness|  tempo|key|time_signature|valence|release_year|followers|release_month|prediction|
+------------+------------+-----------+------+----------------+--------+--------+----+-----------+-------+---+--------------+-------+------------+---------+-------------+----------+
|       0.963|       0.594|   320707.0| 0.407|           0.235|  0.0819|  -9.166| 1.0|     0.0499|179.469|0.0|           4.0|  0.359|      2012.0|    126.0|      January|       5.0|
|     0.00108|       0.695|   195938.0| 0.861|           0.808|  0.0862|  -3.726| 1.0|     0.0282|128.005|0.0|           3.0|  0.504|      2020.0|    184.0|         June|       5.0|
|     0.00287|       0.566|   183080.0| 0.865|         1.86E-4|   0.171|  -9.088| 0.0|    